# Stock Market Data Endpoint - Complete Implementation

This notebook contains the step-by-step implementation for fetching real-time stock market data from Yahoo Finance using Flask API.

## Step 1: Import Required Libraries and Setup

In [2]:
# Import required libraries
from flask import Flask, request, jsonify
import yfinance as yf
import json
from datetime import datetime
import re
import time

print("✅ All libraries imported successfully")
print("📦 Required packages: flask, yfinance")
print("💡 Install with: pip install flask yfinance")

✅ All libraries imported successfully
📦 Required packages: flask, yfinance
💡 Install with: pip install flask yfinance


## Step 2: Create Helper Functions for Symbol Search and Validation

In [3]:
def validate_symbol(symbol):
    """
    Validate if the provided symbol is a valid format
    """
    if not symbol or len(symbol) < 1 or len(symbol) > 10:
        return False
    return symbol.replace('.', '').replace('-', '').isalnum()

def search_company_symbol(company_name):
    """
    Search for company symbol using company name
    Enhanced with more comprehensive mappings
    """
    try:
        # Clean the company name for search
        clean_name = re.sub(r'[^a-zA-Z0-9\s]', '', company_name).strip().upper()

        # Comprehensive company name to symbol mappings
        common_mappings = {
            'APPLE': 'AAPL', 'APPLE INC': 'AAPL',
            'MICROSOFT': 'MSFT', 'MICROSOFT CORPORATION': 'MSFT',
            'GOOGLE': 'GOOGL', 'ALPHABET': 'GOOGL',
            'AMAZON': 'AMZN', 'TESLA': 'TSLA',
            'META': 'META', 'FACEBOOK': 'META',
            'NETFLIX': 'NFLX', 'NVIDIA': 'NVDA',
            'INTEL': 'INTC', 'IBM': 'IBM',
            'ORACLE': 'ORCL', 'SALESFORCE': 'CRM',
            'ADOBE': 'ADBE', 'PAYPAL': 'PYPL',
            'ZOOM': 'ZM', 'SPOTIFY': 'SPOT',
            'WALMART': 'WMT', 'COCA COLA': 'KO',
            'PEPSI': 'PEP', 'JOHNSON AND JOHNSON': 'JNJ',
            'VISA': 'V', 'MASTERCARD': 'MA',
            'DISNEY': 'DIS', 'MCDONALDS': 'MCD',
            'NIKE': 'NKE', 'STARBUCKS': 'SBUX'
        }

        # Check direct mapping first
        if clean_name in common_mappings:
            return common_mappings[clean_name]

        # Check partial matches
        for name_key, symbol in common_mappings.items():
            if clean_name in name_key or name_key in clean_name:
                return symbol

        # If no mapping found, try using the input as a symbol directly
        if len(company_name) <= 5 and company_name.isalpha():
            return company_name.upper()

        return None

    except Exception as e:
        print(f"Error searching for symbol: {str(e)}")
        return None

print("✅ Helper functions created successfully")

✅ Helper functions created successfully


## Step 3: Create Core Stock Market Data Retrieval Function

In [4]:
def get_stock_market_data(company_name):
    """
    Retrieve comprehensive real-time stock market data for a company
    Returns all required metrics as specified in the requirements
    """
    try:
        # Step 3.1: Convert company name to symbol
        symbol = search_company_symbol(company_name)

        if not symbol:
            return {
                'error': f"Could not find symbol for company: {company_name}",
                'suggestion': "Please try entering the stock symbol directly (e.g., AAPL, MSFT)"
            }

        # Step 3.2: Create ticker object and fetch data
        ticker = yf.Ticker(symbol)
        info = ticker.info

        # Step 3.3: Validate data availability
        if not info or len(info) < 5:
            return {
                'error': f"No valid market data found for symbol: {symbol}",
                'tried_symbol': symbol
            }

        # Step 3.4: Extract and format all required market data
        market_data = {
            # Core Identification
            'Symbol': info.get('symbol', symbol.upper()),
            'Company_Name': info.get('longName', info.get('shortName', 'N/A')),
            'Short_Name': info.get('shortName', 'N/A'),
            'Exchange': info.get('exchange', 'N/A'),
            'Currency': info.get('currency', 'USD'),

            # Key Market Status & Price
            'Market_State': info.get('marketState', 'UNKNOWN'),
            'Current_Market_Price': info.get('currentPrice', info.get('regularMarketPrice', 0)),
            'Regular_Market_Price': info.get('regularMarketPrice', 0),
            'Price_Change': info.get('regularMarketChange', 0),
            'Regular_Market_Change': info.get('regularMarketChange', 0),
            'Percentage_Change': info.get('regularMarketChangePercent', 0),
            'Regular_Market_Change_Percent': info.get('regularMarketChangePercent', 0),

            # Daily Performance
            'Previous_Close': info.get('previousClose', 0),
            'Open': info.get('open', info.get('regularMarketOpen', 0)),
            'Day_Low': info.get('dayLow', info.get('regularMarketDayLow', 0)),
            'Day_High': info.get('dayHigh', info.get('regularMarketDayHigh', 0)),

            # Longer-Term Range
            'Fifty_Two_Week_Low': info.get('fiftyTwoWeekLow', 0),
            'Fifty_Two_Week_High': info.get('fiftyTwoWeekHigh', 0),

            # Trading Metrics
            'Volume': info.get('volume', info.get('regularMarketVolume', 0)),
            'Market_Cap': info.get('marketCap', 0),

            # Timestamp
            'Quote_Timestamp': datetime.fromtimestamp(
                info.get('regularMarketTime', time.time())
            ).isoformat() if info.get('regularMarketTime') else datetime.now().isoformat()
        }

        return market_data

    except Exception as e:
        return {
            'error': f"Error retrieving market data: {str(e)}",
            'company_name': company_name
        }

print("✅ Core stock market data function created successfully")

✅ Core stock market data function created successfully


## Step 4: Create Flask Application and API Endpoints

In [5]:
# Initialize Flask application
app = Flask(__name__)
app.config['JSON_SORT_KEYS'] = False

@app.route('/api/stock/market-data', methods=['POST'])
def get_stock_market_data_endpoint():
    """
    API endpoint to retrieve real-time stock market data by company name
    """
    try:
        data = request.get_json()

        if not data or 'company_name' not in data:
            return jsonify({
                'error': 'Missing company_name',
                'message': 'Request body must contain a "company_name" field'
            }), 400

        company_name = data['company_name'].strip()

        if not company_name:
            return jsonify({
                'error': 'Empty company name',
                'message': 'Company name cannot be empty'
            }), 400

        # Get stock market data
        market_data = get_stock_market_data(company_name)

        # Check if there was an error
        if 'error' in market_data:
            return jsonify(market_data), 404

        return jsonify({
            'success': True,
            'data': market_data,
            'timestamp': datetime.now().isoformat()
        }), 200

    except Exception as e:
        return jsonify({
            'error': 'Internal server error',
            'message': str(e)
        }), 500

@app.route('/api/stock/market-data/<symbol>', methods=['GET'])
def get_stock_market_data_by_symbol_endpoint(symbol):
    """
    API endpoint to retrieve real-time stock market data by symbol
    """
    try:
        # Validate symbol format
        if not validate_symbol(symbol):
            return jsonify({
                'error': 'Invalid symbol format',
                'message': 'Symbol must be 1-10 characters, alphanumeric with optional dots or hyphens'
            }), 400

        # Get stock market data using symbol directly
        market_data = get_stock_market_data(symbol)

        if 'error' in market_data:
            return jsonify(market_data), 404

        return jsonify({
            'success': True,
            'data': market_data,
            'timestamp': datetime.now().isoformat()
        }), 200

    except Exception as e:
        return jsonify({
            'error': 'Internal server error',
            'message': str(e)
        }), 500

print("✅ Flask API endpoints created successfully")

✅ Flask API endpoints created successfully


## Step 5: Add Interactive User Input Function

In [6]:
def interactive_stock_lookup():
    """
    Interactive function that asks user for company name and displays market data
    """
    print("🔍 Stock Market Data Lookup")
    print("=" * 40)

    while True:
        try:
            # Ask user for company name
            company_name = input("\n📊 Enter the company name (or 'quit' to exit): ").strip()

            if company_name.lower() in ['quit', 'exit', 'q']:
                print("👋 Goodbye!")
                break

            if not company_name:
                print("❌ Please enter a valid company name.")
                continue

            print(f"\n🔄 Fetching market data for: {company_name}...")

            # Get market data
            market_data = get_stock_market_data(company_name)

            if 'error' in market_data:
                print(f"❌ Error: {market_data['error']}")
                if 'suggestion' in market_data:
                    print(f"💡 {market_data['suggestion']}")
                continue

            # Display formatted results
            print("\n" + "=" * 60)
            print(f"📈 MARKET DATA FOR {market_data['Company_Name'].upper()}")
            print("=" * 60)

            print("\n🏢 CORE IDENTIFICATION:")
            print(f"   Symbol: {market_data['Symbol']}")
            print(f"   Company Name: {market_data['Company_Name']}")
            print(f"   Short Name: {market_data['Short_Name']}")
            print(f"   Exchange: {market_data['Exchange']}")
            print(f"   Currency: {market_data['Currency']}")

            print("\n💰 KEY MARKET STATUS & PRICE:")
            print(f"   Market State: {market_data['Market_State']}")
            print(f"   Current Price: ${market_data['Current_Market_Price']:.2f}")
            print(f"   Regular Market Price: ${market_data['Regular_Market_Price']:.2f}")
            print(f"   Price Change: ${market_data['Price_Change']:.2f}")
            print(f"   Percentage Change: {market_data['Percentage_Change']:.2f}%")

            print("\n📊 DAILY PERFORMANCE:")
            print(f"   Previous Close: ${market_data['Previous_Close']:.2f}")
            print(f"   Open: ${market_data['Open']:.2f}")
            print(f"   Day Low: ${market_data['Day_Low']:.2f}")
            print(f"   Day High: ${market_data['Day_High']:.2f}")

            print("\n📈 LONGER-TERM RANGE:")
            print(f"   52 Week Low: ${market_data['Fifty_Two_Week_Low']:.2f}")
            print(f"   52 Week High: ${market_data['Fifty_Two_Week_High']:.2f}")

            print("\n📋 TRADING METRICS:")
            print(f"   Volume: {market_data['Volume']:,}")
            print(f"   Market Cap: ${market_data['Market_Cap']:,}")

            print("\n⏰ TIMESTAMP:")
            print(f"   Quote Time: {market_data['Quote_Timestamp']}")

            print("\n" + "=" * 60)

        except KeyboardInterrupt:
            print("\n\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"❌ Unexpected error: {str(e)}")

print("✅ Interactive lookup function created successfully")

✅ Interactive lookup function created successfully


## Step 6: Test the Implementation

In [7]:
# Test the stock market data function with a sample company
print("🧪 Testing Stock Market Data Function")
print("=" * 50)

# Test with Apple Inc
test_company = "Apple"
print(f"\n📊 Testing with: {test_company}")

result = get_stock_market_data(test_company)

if 'error' in result:
    print(f"❌ Error: {result['error']}")
else:
    print("✅ Success! Sample data retrieved:")
    print(f"   Company: {result['Company_Name']}")
    print(f"   Symbol: {result['Symbol']}")
    print(f"   Current Price: ${result['Current_Market_Price']:.2f}")
    print(f"   Market State: {result['Market_State']}")
    print(f"   Volume: {result['Volume']:,}")

print("\n🎯 Test completed!")

🧪 Testing Stock Market Data Function

📊 Testing with: Apple
✅ Success! Sample data retrieved:
   Company: Apple Inc.
   Symbol: AAPL
   Current Price: $236.06
   Market State: REGULAR
   Volume: 15,434,351

🎯 Test completed!


## Step 7: Run Interactive Demo

In [ ]:
# Run the interactive stock lookup


interactive_stock_lookup()

print("💡 To run the interactive demo, uncomment the line above and run this cell")
print("🚀 The demo will ask you to enter company names and display their market data")

🔍 Stock Market Data Lookup

📊 Enter the company name (or 'quit' to exit): ibm

🔄 Fetching market data for: ibm...

📈 MARKET DATA FOR INTERNATIONAL BUSINESS MACHINES CORPORATION

🏢 CORE IDENTIFICATION:
   Symbol: IBM
   Company Name: International Business Machines Corporation
   Short Name: International Business Machines
   Exchange: NYQ
   Currency: USD

💰 KEY MARKET STATUS & PRICE:
   Market State: REGULAR
   Current Price: $256.77
   Regular Market Price: $256.77
   Price Change: $0.68
   Percentage Change: 0.27%

📊 DAILY PERFORMANCE:
   Previous Close: $256.09
   Open: $256.12
   Day Low: $254.88
   Day High: $257.69

📈 LONGER-TERM RANGE:
   52 Week Low: $202.87
   52 Week High: $296.16

📋 TRADING METRICS:
   Volume: 1,554,016
   Market Cap: $239,186,116,608

⏰ TIMESTAMP:
   Quote Time: 2025-09-09T16:14:12

